<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2022_GoogleColabs/blob/main/Week3/SumGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
!stat cuda
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

/usr/local
  File: cuda -> /usr/local/cuda-10.1
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 24h/36d	Inode: 106         Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-01-10 17:12:54.298681814 +0000
Modify: 2022-01-10 17:12:54.189681484 +0000
Change: 2022-01-10 17:12:54.189681484 +0000
 Birth: -
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-xqon44ki
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-xqon44ki
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=5abdc954d328a2764b1ea45165cddc62c83bd3af90cf78f579f4f2e2d30acd71
  Stored in directory: /tmp/pip-ephem-wheel-cache-xu8_ilkn/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin
directory /usr/local/src already exists
Out bin /usr/local/result.out


In [8]:
%%cu
#include <stdio.h>

__global__ void sum(int* a, int* b, int* c, const int N) {
	int i = threadIdx.x + blockDim.x * blockIdx.x;
	if (i<N)
		c[i] = a[i] + b[i];	
	//else 
	//	printf("i > N -> %3d in block: %d\n", i, blockIdx.x);
}

void sum_host(int* a, int* b, int* c, const int N) {
	for(int i=0; i<N; ++i)
		c[i] = a[i] + b[i];	
}


int main() { 
	int* h_a = 0;
	int* h_b = 0;
	int* h_c = 0;
	
	int* d_a = 0;
	int* d_b = 0;
	int* d_c = 0;

	const int N = 2048;
	int numThreadsPerBlock =  128;
	int numBlocksPerGrid = ceilf(N/numThreadsPerBlock + 1);
	printf("Num threads per block: %3d\n", numThreadsPerBlock);
	printf("Num blocks per grid: %3d\n", numBlocksPerGrid);

	size_t size = N * sizeof(int);

	//allocate host memory
	h_a = (int*)malloc(size);	 
	h_b = (int*)malloc(size);	 
	h_c = (int*)malloc(size);	 

	//initialize a, b and c
	for(int i=0;i<N;++i) {
		h_a[i] = i+1;
		h_b[i] = h_a[i]*2;
		h_c[i] = 0;
	}

	//allocate device memory 
	cudaMalloc((void**)&d_a, size);
	cudaMalloc((void**)&d_b, size);
	cudaMalloc((void**)&d_c, size);
	
	//copy host data to device memory
	cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice); 
	cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

	//calculate on host 
	sum_host(h_a, h_b, h_c, N); 
	printf("Sum (host): %d\n", h_c); 
  
	//output result
	/*
	printf("Host calculation result: \n");
	for(int i=0;i<N;++i) {
		printf("%3d + %3d = %3d\n", h_a[i], h_b[i], h_c[i]);
		//clear host result to ensure that the result of device is actually from the kernel
		h_c[i] = 0;
	}
	*/

	//calculate on device
	sum<<<numBlocksPerGrid, numThreadsPerBlock>>>(d_a, d_b, d_c, N);

	//copy result from device to host
	cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

  printf("Sum (device): %d\n", h_c);

	//output result
	/*
	printf("--------------------------------------\n");
	printf("Device calculation result: \n");
	for(int i=0;i<N;++i) {
		printf("%3d + %3d = %3d\n", h_a[i], h_b[i], h_c[i]);
	}
	printf("--------------------------------------\n");
	*/
 

	//delete data allocated on device
	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	//delete host memory
	free(h_a);
	free(h_b);
	free(h_c);

	cudaDeviceReset();
	return 0;
}

Num threads per block: 128
Num blocks per grid:  17
Sum (host): 1877032960
Sum (device): 1877032960

